These code are belong to https://github.com/nessessence
feel free to distribute the code for education purpose 

DFS: O(V+E)
    - implemented by using recursion or stack
    - use less memory tha BFS
    - the first founded solution state is not guaranteed to be the nearest solution to the root

In [2]:
# Detecting Cycle: Undirected Graph
def detect_cycle(G,nv,ne):
    def dfs(v,p,G,Visited):
        Visited[v] = True
        for child in G[v]:
            if child != p:
                if Visited[child] : return True
                if dfs(child,v,G,Visited): return True
        return False
    if ne > nv-1: return True #upperbound: for undirected graph: atmost: "Tree"
    Visited = [ False for i in range(nv)]
    for v in range(nv):
        if not Visited[v]:
            if dfs(v,-1,G,Visited) : return True
    return False
        
nG = int(input())
for _ in range(nG):
    nv,ne = [int(e) for e in input().split()]
    G = { v:[] for v in range(nv)}
    for _ in range(ne):
        a,b = [int(e) for e in input().split()]
        G[a].append(b); G[b].append(a)
    if detect_cycle(G,nv,ne): print("YES")
    else: print("NO")

In [ ]:
# Cycle in Tree : find a cycle length in Tree
from sys import stdin as kb
def cycle_len(nv,G):
    def dfs(v,p,d,Distance,G):
        Distance[v] = d
        for child in G[v]:
            if child == p: continue
            if Distance[child] != -1: return Distance[v] - Distance[child] + 1
            dis = dfs(child,v,d+1,Distance,G)
            if dis > 0 : return dis
        return 0
    Distance = [ -1 for i in range(nv) ]
    for v in range(nv):
        return dfs(v,-1,0,Distance,G) # it just tree with one more edge 
nv = int(input())
G = { v : [] for v in range(nv) }
for _ in range(nv):
    a,b = [int(e) for e in kb.readline().split()]
    G[a].append(b); G[b].append(a)
print(cycle_len(nv,G))


Topological Sort:
    - "applying dfs" to get an array with non-fan-in order (eg application : prerequisite scheduling )
    - concept: the result is sorting the ending time of applying DFS in ascending order 

In [ ]:
# Task Planning: topological sort
# implement 1: using Stack
def topological_sort(n,Pre):
    Stack = []; Visited = [False]*n 
    def dfs(v,Visited,G):
        Visited[v] = True
        for child in G[v]:
            if not Visited[child]:
                dfs(child,Visited,G)
        Stack.append(v) # queue --> will be reverse 
    for v in range(n):
        if not Visited[v]: dfs(v,Visited,Pre)
    return Stack[::-1] #reverse queue --> stack like
n = int(input())
Prerequisites = { v : [] for v in range(n)   }
for i in range(n):
    # make directed graph
    for pre in [int(e) for e in input().split()][1:] : Prerequisites[pre].append(i)
print( ' '.join(map(str,topological_sort(n,Prerequisites))))

# Implement 2: using accumulator 
def topo(): 
    def dfs(v,p,n,G,Begin,End):
        Begin[v] = n
        for child in G[v]:
            if child != p and Begin[child] == -1:
                n =  dfs(child,v,n+1,G,Begin,End)
        End[v] = n+1
        return n+1
    n_task = int(input())
    G = {task:[] for task in range(n_task)}
    for task in range(n_task):
        for pre in list(map(int,input().split()))[1:] : G[pre].append(task)
    Begin = [ -1 for i in range(n_task) ]
    End = [ -1 for i in range(n_task) ]
    n_count = 0
    for task in range(n_task):
        if Begin[task] == -1:
            n_count =  dfs(task,-1,n_count,G,Begin,End)
    non_prerequisite_ordered = [task for end,task in sorted( [(end,i) for i,end in enumerate(End)], reverse=True ) ]
    print(' '.join(map(str, non_prerequisite_ordered)) )   
topo()

Strongly Connected Component: O(V+E) 
- apply topological sort algorithm on G_T ( inverse direction of G) (not exactly topological order, because cycles are allowed )
- then apply dfs to the previous result

In [ ]:
# Catapult:
# implemetation I
def SCC(nv,G,G_T):
    Stack = list(); Visited = [False]*nv
    def topo(v,Visited,G_T):
        Visited[v] = True
        for child in G_T[v]:
            if not Visited[child]:
                topo(child,Visited,G_T)
        Stack.append(v)
    def dfs_count(v,n,Visited,G):
        Visited[v] = True
        for child in G[v]:
            if not Visited[child]:
                n = dfs_count(child,n,Visited,G)
        return n+1 # n is just an accumulator 
    for v in range(nv):
        if not Visited[v]: topo(v,Visited,G_T)
    order_to_traverse = Stack[::-1]
    Visited = [False]*nv;
    connected_component = []
    for v in order_to_traverse:
        if not Visited[v]:
            connected_component.append(dfs_count(v,0,Visited,G))
    return sorted(connected_component) # require: ascending output
nv = int(input())
G = { v : [] for v in range(nv) }; G_T = { v : [] for v in range(nv) }
for v in range(nv):
    l = list(map(int,input().split()))[1:]
    G[v] = l
    for u in l: G_T[u].append(v)
print( ' '.join(map(str,SCC(nv,G,G_T))))

# implementation II 
def topological_sort(G,n):
    def dfs_topo(v,G,p,n,Begin,End):
        Begin[v] = n
        for child in G[v]:
            if child != p and Begin[child] == -1:
                n = dfs_topo(child,G,v,n+1,Begin,End)
        End[v] = n+1
        return n+1
    End = [ -1 for i in range(n)]; Begin = [ -1 for i in range(n)]
    n = 0 
    for v in G:
        if End[v] == -1:  n = dfs_topo(v,G,-1,n,Begin,End)
    return  sorted([ (end,v) for v,end in enumerate(End) ],reverse=True)
def SCC(G,G_T,n):
    def dfs_scc(v,G,p,Visited):
        Visited[v] = True
        SCC.nV += 1
        for child in G[v]:
            if not Visited[child] and child != p:
                dfs_scc(child,G,v,Visited)
                
    topo_sorted_GT = topological_sort(G_T,n)
    Visited = [ False for i in range(n) ]
    ans = [] 
    for end,v in topo_sorted_GT:
        if not Visited[v]:
            SCC.nV = 0
            dfs_scc(v,G,-1,Visited)
            ans.append(SCC.nV)
    return sorted(ans)
    
n = int(input())
G = {}; G_T = {v:[] for v in range(n)}
for i in range(n):
    l = [int(e) for e in input().split() ]
    G[i] = l[1:]
    for v in l[1:]: G_T[v].append(i)
ans = SCC(G,G_T,n)
for nV in ans: print(nV,end=" ")

In [ ]:
# IKEA : topological sort
# just checking the prerequisite 
# this can be called as Kahn algorithm
from sys import stdin as kb
def pre_check(query,pre):
  so_far = set()
  for q in query:
    if not pre[q].issubset(so_far) : return "FAIL"   # *check_subset
    so_far.add(q)
  return "SUCCESS"
nv, ne = list(map(int,kb.readline().split()))
pre = {v : set() for v in range(1,nv+1)}
for i in range(ne):
    a,b = list(map(int,kb.readline().split()))
    pre[b].add(a)
for i in range(5):
    query = list(map(int,kb.readline().split() ) )
    print(pre_check(query,pre))

In [ ]:
# connected count 
import sys
sys.setrecursionlimit(1000000)
def component_count(nv,G):
    def dfs(v,Visited,G):
        Visited[v] = True
        for child in G[v]:
            if Visited[child]: continue
            dfs(child,Visited,G)
    Visited = [ False for _ in range(nv+1) ]
    count = 0
    for v in range(1,nv+1):
        if not Visited[v]:
            count+=1
            dfs(v,Visited,G)
    return count
nv, ne = [int(e) for e in input().split() ]
G = {v:[] for v in range(nv+1)}
for _ in range(ne):
    a,b = [int(e) for e in input().split()]
    G[a].append(b); G[b].append(a)
print(component_count(nv,G))


In [ ]:
# line graph: linear graph (**no cycle + n0 more than 2 edge per node)
def count_linear_graph(nv,G):
    def dfs(v,p,Visited,G):
        Visited[v] = True
        T = len(G[v]) <= 2
        for child in G[v]:
            if p == child: continue
            if Visited[child] : return False
            T = T and dfs(child,v,Visited,G)
        return T
    Visited = [ False for _ in range(nv) ]
    count = 0
    for v in range(nv):
        if not Visited[v] and dfs(v,-1,Visited,G): count += 1
    return count 
nv, ne = [int(e) for e in input().split()]
G = { v : [] for v in range(nv) }
for _ in range(ne):
    a,b = [int(e) for e in input().split() ]
    G[a].append(b); G[b].append(a)
print(count_linear_graph(nv,G))


BFS: O(V+E)
- use Queue instead of stack
- comsume more memory (might not be practical to use in some cases)
- BFS is usually used to deal with problems related with path length 
    - especially for shortest path problem: because when the first the solution state will definitely be the answer 

In [ ]:
# connection
 Cycle in Tree : find a cycle length in Tree
def connected_count(G,k,nv):
    def bfs(v,G):
        Queue = list([(v,0)])
        Visited = [False for i in range(nv)]
        Visited[v] = True 
        total_connection = 0
        while len(Queue) != 0:
            v,degree = Queue.pop(0)
            if degree > k: return total_connection
            total_connection += 1 
            for child in G[v]:
                if Visited[child] : continue
                Visited[child] = True
                Queue.append((child,degree+1))
        return total_connection
    max_total_connection = 0
    for v in range(nv): max_total_connection = max(max_total_connection,bfs(v,G))
    return max_total_connection
nv,ne,k = [int(e) for e in input().split()]
G = { v : [] for v in range(nv) }
for _ in range(ne):
    a,b = [int(e) for e in input().split()]
    G[a].append(b); G[b].append(a)
print(connected_count(G,k,nv))


In [ ]:
# perimeter
def perimeter(nv,k,G):
    def bfs(nv,k,G):
        Queue = list([(0,0)])
        Visited = [True] + [False for i in range(nv)]
        total_location = 0
        while len(Queue) > 0:
            v,distance =  Queue.pop(0)
            if distance == k: total_location += 1
            if distance > k: return total_location
            for child in G[v]:
                if Visited[child] : continue
                Visited[child] = True
                Queue.append((child,distance+1))
        return total_location
    return bfs(nv,k,G)
nv,ne,k = [int(e) for e in input().split()]
G = { v : [] for v in range(nv+1) }
for _ in range(ne):
    a,b = [int(e) for e in input().split()]
    G[a].append(b); G[b].append(a)
print(perimeter(nv,k,G))

In [ ]:
# pandemic
def pandemic(r,c,nday,T):
    total_infected_cities = set( (i,j) for i in range(r) for j in range(c) if T[i][j] == 1) #init
    for n in range(nday):
        if n == 0 : new_infected_cities = list(total_infected_cities)
        infected_list = set()
        for (i,j) in new_infected_cities:
            for x,y in [ (i-1,j),(i+1,j),(i,j-1),(i,j+1) ] :
                if x >= r or x < 0 or y >= c or y < 0 : continue
                if T[x][y] == 2 : continue                    
                infected_list.add((x,y))
                total_infected_cities.add((x,y))
        new_infected_cities = list(infected_list)
    return len(total_infected_cities)       
r,c,nday = [int(e) for e in input().split() ]
T = [ list(map(int,input().split())) for i in range(r) ]
print(pandemic(r,c,nday,T))


Minimum Spanning Tree:
- Prim algorithm: 
    - greedy algorithm
    - use greedy to select node with minimum distance(weight) to the current MST 
        - (MST start with one randomly selected node)
    - O(v^2) or O(elogv) by using min_heap
- Kruskal algorithm:
    - greedy algotithm
    - use greedy to select the minimum weight edge (which both nodes are not already in the same disjoin set)
    - use disjoint set to check the MST group and use union to connect the group to bigger MST (end up with one MST)
    - O(v^2) or O(elogv) by using min_heap

In [ ]:
# Cable Company 
import heapq
def Prim(nv,Cost):
    inMST = [ False for i in range(nv)]
    Distance = [ float('inf') for i in range(nv) ]
    Distance[0] = 0
    PQ = [(0,0)]
    while len(PQ) > 0:
        cost,u = heapq.heappop(PQ)
        inMST[u] = True
        for v in range(nv):
            if not inMST[v] and Cost[(min(u,v),max(u,v))] < Distance[v]:
                Distance[v] = Cost[(min(u,v),max(u,v))]
                heapq.heappush(PQ,(Cost[(min(u,v),max(u,v))],v)) # this code dont remove the longer one, (it will not affect the result, because the longer one cant change anything which the shorter one change before)
    return sum(Distance)
nv = int(input())
Cost = { (i,j+i+1) : cost for i in range(nv-1) for j,cost in enumerate(list(map(int,input().split()))) }
print(Prim(nv,Cost))



n = int(input())
W = {}
W.update({ (a,a+i+1) : b for a in range(n-1) for i,b in enumerate(list(map(int,input().split()))) } )
Tdistance = { v : float('inf') for v in range(n) }; Tdistance[0] = 0
pq = set( (td,v) for v, td in Tdistance.items() ) # Python set is implemented using hash_map (like dict)
inMST = [False for i in range(n) ]
total_cost = 0; nMST = 0
while( nMST != n and len(pq)!=0 ):
    td,v = min(pq)  # O(n)
print(Prim(n,L))
    pq.remove((td,v)) 
    inMST[v] = True
    total_cost += td; nMST += 1 
    for u in range(n):
        if not inMST[u] and W[(min(u,v),max(u,v))] <  Tdistance[u]:
            pq.remove((Tdistance[u],u)) # remove the longer one
            Tdistance[u] = W[(min(u,v),max(u,v))]
            pq.add((Tdistance[u],u))
print(total_cost)

In [ ]:
# XOR_roadL: longest path 
import heapq
from sys import stdin 
def Prim(n,L):
    Distance = [ -float('inf') ] * n
    inMST = [ False ]*n; nv = 0 
    Distance[0] = 0;  PQ = [ (0,0) ]  # d,u
    while len(PQ) > 0 :
        d,u = heapq.heappop(PQ)
        if inMST[u]: continue #*** the same ** as below
        #if abs(d) < Distance[u]: continue # this will reduce redundant, since we dont remove the older shorter node
        inMST[u] = True
        for v in range(n):
            if not inMST[v]: 
                cost = L[u]^L[v] 
                if Distance[v] < cost :
                  heapq.heappush(PQ,(-cost,v))
                  Distance[v] = cost
    return sum(Distance)
n = int(input())
L = [ int(stdin.readline()) for _ in range(n) ]
print(Prim(n,L))


single shortest path:
    - Dijkstra algorithm: Faster
        - can use only for non-negative weight graph
        - like Prim in MST: 
            - no inMST[]
            - use accumulate distance instead of just edge weight
        - if sparse graph : use adjacency list and min_heap : O(elog(v))
        - if dense graph : use adjacency matrix ( find min in O(v) ) : O(v2)
    - Bellman-Ford: Easier
        - can use even for negative weight edge 
        - easy to implement: just 2 for-loop
        - O(ve)  
How to select:
    - no negative weight: ---> Dijkstra
        - sparse graph: ---> adjacency list
        - dense graph: ---> adjacency matrix 
    - otherwise: ---> Bellman Ford

In [ ]:
# Shortest Path with Negative Edge: single shortest path with negative edge ---> Bellman-Ford : O(ve)
def bellman_ford(s,nv,G):
    Distance = [float('inf')]*nv
    Distance[s] = 0
    for i in range(nv):
        for u in range(nv):
            for v,w in enumerate(G[u]):
                if i == nv-1 and  Distance[u]+w < Distance[v] : return [-1]
                Distance[v] = min( Distance[v], Distance[u]+w)
    return Distance
nv, ne, s = [int(e) for e in input().split() ]
G = {v : [float('inf')]*nv for v in range(nv)}
for _ in range(ne):
    a,b,w = [int(e) for e in input().split() ]
    G[a][b] = w
print(' '.join(map(str,bellman_ford(s,nv,G))))

In [ ]:
# Hole : shortest path: djisktra ( can view as state space search)
from sys import stdin
import heapq
def dijkstra():
    n,a,b = [int(e) for e in stdin.readline().split() ]
    Holes = set( (i-1,j-1) for _ in range(n) for i,j in [list(map(int,stdin.readline().split()))])
    nHoles = [ [float('inf')]*1000 for _ in range(1000) ]
    nHoles[a-1][b-1] = 0
    PQ = [(0,a-1,b-1)]
    while len(PQ) > 0 :
        nhole, a,b = heapq.heappop(PQ)
        if nhole > nHoles[a][b] : continue   
        if a == 0 or a == 999 or b == 0 or b == 999: return nhole  # definitely
        for x,y in [(a+1,b),(a-1,b),(a,b+1),(a,b-1)]:
            if (x,y) in Holes and nhole+1 < nHoles[x][y] :
                nHoles[x][y] = nhole+1
                heapq.heappush(PQ,((nhole+1,x,y)))
            elif (x,y) not in Holes and  nhole < nHoles[x][y]:
                nHoles[x][y] = nhole
                heapq.heappush(PQ,((nhole,x,y)))
print(dijkstra())

In [ ]:
# Columbia 
import heapq
def dijkstra(r,c,Cost):
    Distance = [ [float('inf')]*c for _ in range(r) ]
    Distance[0][0] = 0 # init the start point
    MinHeap = [(0,0,0)]  # d,x,y
    while len(MinHeap) > 0:
        d,x,y = heapq.heappop(MinHeap)
        for i,j in [ (x-1,y),(x+1,y),(x,y-1),(x,y+1) ]:
            if i < 0 or i >= r or j < 0 or j >= c: continue
            if  d + Cost[i][j] < Distance[i][j]:
                heapq.heappush(MinHeap,(d + Cost[i][j],i,j))
                Distance[i][j] = d + Cost[i][j]
    return Distance
r,c = [int(e) for e in input().split() ]
Cost = [ list(map(int,input().split())) for _ in range(r) ]
print('\n'.join(map(str,dijkstra(r,c,Cost))).replace(',','').replace(']','').replace('[','') ) # fast printing of 2D list


min_graph_coloring: state space search (NP-complete)

In [ ]:
# min_graph_coloring: graph: state_space_search
# implementation I
import heapq
def min_GraphColoring(nv,G):
    def valid_coloring(v,c,colors,nv,G):
        for u in G[v]:
            if colors[u] == c: return False
        return True
    colors = [-1]*nv
    minHeap = [ (1,colors,0)] # n_color, colors_assignment, index
    while len(minHeap) > 0:
        n_color, colors, i = heapq.heappop(minHeap)
        if i == nv : return n_color
        for c in range(n_color):
            if valid_coloring(i,c,colors,nv,G):
                new_colors = colors[:]
                new_colors[i] = c  
                heapq.heappush(minHeap, (n_color,new_colors,i+1) )
                heapq.heappush(minHeap,(n_color+1,new_colors[:],i+1))
nv,ne = list(map(int,input().split()))
G = { v : [] for v in range(nv) }
for _ in range(ne):
    a,b = list(map(int,input().split()))
    G[a].append(b); G[b].append(a)
print(min_GraphColoring(nv,G))


# impementation II 
from queue import PriorityQueue
def grap_coloring():
    def valid_list_color(c,i,list_color,G):
        for v in G[i]:
            if c == list_color[v] : return False
        return True
    PQ = PriorityQueue()
    PQ.put([ 1,[-1]*nv,0 ])
    while not PQ.empty() :
        k,list_color,i = PQ.get()
        if i == nv: return k
        for c in range(k):
            if not valid_list_color(c,i,list_color,G): continue
            new_list_color = list_color[:]
            new_list_color[i] = c
            PQ.put( [k, new_list_color, i+1 ] )
        new_k_list_color = list_color[:]
        new_k_list_color[i] = k
        PQ.put( [k+1,new_k_list_color, i+1] )
            
    return False
nv,ne = list(map(int,input().split()))
G = { v : [] for v in range(nv) }
for i in range(ne):
    a,b = list(map(int,input().split()))
    G[a].append(b); G[b].append(a)
print(grap_coloring())

